In [1]:
#importing useful libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.stats as s
from datetime import datetime, date, timedelta
import math

In [2]:
#read COVID-19 data 
Covid_Data = pd.read_csv('merged_df.csv',usecols=[0,2,4,17,18,19,20,21])
Covid_Data.rename(columns={'Difference_Yesterday':'PCR_Postive_Count','Hospitalized_diff':'PCR_Positive_Hospitalised','Death_diff':'Deceased'}, inplace = True)
#data.drop([0,1], axis=0, inplace=True)
Covid_Data['PCR_Positive_Hospitalised'] = Covid_Data['PCR_Positive_Hospitalised'].fillna(0).astype('int64')
#Covid_Data

In [3]:
Covid_Data1 = Covid_Data
Covid_Data1['Date']=pd.to_datetime(Covid_Data['Date'].astype(str), format='%d-%m-%Y')

#Covid_Data['Date']= pd.to_datetime(Covid_Data['Date'].datetime.strftime('%m-%d-%Y'))
#Covid_Data['Date'] = Covid_Data['Date'].datetime.strftime('%m-%d-%Y')
#print(Covid_Data['Date'])


In [4]:
#Autumn
#Aut_mon = ['2021KW39','2021KW40','2021KW41','2021KW42','2021KW43','2021KW44','2021KW45','2021KW46','2021KW47']
end_dt_aut = '28-11-2021'
data_autumn = Covid_Data1.loc[Covid_Data['Date'] <= pd.to_datetime(end_dt_aut, format='%d-%m-%Y')]
#data_autumn.to_csv('result.csv')

#Winter
"""
Win_mon = ['2021KW48','2021KW49','2021KW50','2021KW51','2021KW52','2022KW01','2022KW02','2022KW03','2022KW04','2022KW05','2022KW06','2022KW07','2022KW08']
data_winter = data[data['Week'].isin(Win_mon)]
"""
str_dt_win = '29-11-2021'
end_dt_win = '27-02-2022'
data_winter = Covid_Data1.loc[(Covid_Data1['Date'] >= pd.to_datetime(str_dt_win, format='%d-%m-%Y')) & (Covid_Data1['Date'] <= pd.to_datetime(end_dt_win, format='%d-%m-%Y'))]
#data_winter.to_csv('result1.csv')
#data_autumn.tail(20)

#Spring
"""
Spr_mon = ['2022KW09','2022KW10','2022KW11','2022KW12','2022KW13','2022KW14','2022KW15','2022KW16','2022KW17','2022KW18','2022KW19','2022KW20']
data_spring = data[data['Week'].isin(Spr_mon)]
"""
str_dt_spr = '28-02-2022'
data_spring = Covid_Data1.loc[(Covid_Data1['Date'] >= pd.to_datetime(str_dt_spr, format='%d-%m-%Y'))]
#data_spring.to_csv('result2.csv')
#data_autumn.tail(20)


In [5]:
#printing dataframes
#data_autumn.head(10)
#data_autumn.tail(10)

#data_winter.head(10)
#data_winter.tail(10)

#data_spring.head(10)
#data_spring.tail(10)

In [6]:
"""
Correlation:
1. Positive: when one variable moves higher or lower, the other variable moves in the same direction with the same magnitude.
2. Negative: if one variable increases, the other variable decreases with the same magnitude (and vice versa).
"""

'\nCorrelation:\n1. Positive: when one variable moves higher or lower, the other variable moves in the same direction with the same magnitude.\n2. Negative: if one variable increases, the other variable decreases with the same magnitude (and vice versa).\n'

In [7]:
#correlation matrix --- Kendall

def find_cor_sig(data,final_list,season_name):
    covid_param = ['PCR_Postive_Count','PCR_Positive_Hospitalised','Deceased']
    weather_param = ['temperature','wind_speed','relative_humidity']
    return_list = []
    for i in range(0,len(covid_param)):
        for j in range(0,len(weather_param)):
            corr_kendall= s.kendalltau(data[covid_param[i]],data[weather_param[j]])
            corr = round(corr_kendall[0],4)
            sig = corr_kendall[1] <= 0.05
            return_list.append(('Kendall-Tau',season_name,covid_param[i],weather_param[j],corr,corr_kendall[1],sig))
    for i in range(0,len(covid_param)):
        for j in range(0,len(weather_param)):
            corr_spear= s.spearmanr(data[covid_param[i]],data[weather_param[j]])
            corr = round(corr_spear[0],4)
            sig = corr_spear[1] <= 0.05
            return_list.append(('Spearmann',season_name,covid_param[i],weather_param[j],corr,corr_spear[1],sig))        
        
    #corr_kendall1= data.corr(method='kendall')
    for val in range(0,len(return_list)):
        final_list.append(return_list[val])
    return final_list

In [8]:
#corr_kendall_aut1.style.background_gradient(cmap='Oranges')
#----------autumn months correlation--------------  
final_list = []
final_list1 = find_cor_sig(data_autumn,final_list,'Autumn')
final_list2 = find_cor_sig(data_winter,final_list1,'Winter')
final_list3 = find_cor_sig(data_spring,final_list2,'Spring')

corr_sig_data = pd.DataFrame (final_list3, columns = ['Correlation Method','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_sig_data.to_csv('corr_sig_data.csv')